# Project 6: Analyzing Stock Sentiment from Twits
## Instructions
Each problem consists of a function to implement and instructions on how to implement the function.  The parts of the function that need to be implemented are marked with a `# TODO` comment.

## Packages
When you implement the functions, you'll only need to you use the packages you've used in the classroom, like [Pandas](https://pandas.pydata.org/) and [Numpy](http://www.numpy.org/). These packages will be imported for you. We recommend you don't add any import statements, otherwise the grader might not be able to run your code.

### Load Packages

In [1]:
import json
import nltk
import os
import random
import re
import torch
from tqdm import tqdm
import numpy as np
import pandas as pd

from torch import nn, optim
import torch.nn.functional as F

## Introduction
When deciding the value of a company, it's important to follow the news. For example, a product recall or natural disaster in a company's product chain. You want to be able to turn this information into a signal. Currently, the best tool for the job is a Neural Network. 

For this project, you'll use posts from the social media site [StockTwits](https://en.wikipedia.org/wiki/StockTwits). The community on StockTwits is full of investors, traders, and entrepreneurs. Each message posted is called a Twit. This is similar to Twitter's version of a post, called a Tweet. You'll build a model around these twits that generate a sentiment score.

We've collected a bunch of twits, then hand labeled the sentiment of each. To capture the degree of sentiment, we'll use a five-point scale: very negative, negative, neutral, positive, very positive. Each twit is labeled -2 to 2 in steps of 1, from very negative to very positive respectively. You'll build a sentiment analysis model that will learn to assign sentiment to twits on its own, using this labeled data.

The first thing we should to do, is load the data.

## Import Twits 
### Load Twits Data 
This JSON file contains a list of objects for each twit in the `'data'` field:

```
{'data':
  {'message_body': 'Neutral twit body text here',
   'sentiment': 0},
  {'message_body': 'Happy twit body text here',
   'sentiment': 1},
   ...
}
```

The fields represent the following:

* `'message_body'`: The text of the twit.
* `'sentiment'`: Sentiment score for the twit, ranges from -2 to 2 in steps of 1, with 0 being neutral.


To see what the data look like by printing the first 10 twits from the list. 

In [2]:
with open(os.path.join('..', '..', 'data', 'project_6_stocktwits', 'twits.json'), 'r') as f:
    twits = json.load(f)

print(twits['data'][:10])

[{'message_body': '$FITB great buy at 26.00...ill wait', 'sentiment': 2, 'timestamp': '2018-07-01T00:00:09Z'}, {'message_body': '@StockTwits $MSFT', 'sentiment': 1, 'timestamp': '2018-07-01T00:00:42Z'}, {'message_body': '#STAAnalystAlert for $TDG : Jefferies Maintains with a rating of Hold setting target price at USD 350.00. Our own verdict is Buy  http://www.stocktargetadvisor.com/toprating', 'sentiment': 2, 'timestamp': '2018-07-01T00:01:24Z'}, {'message_body': '$AMD I heard there’s a guy who knows someone who thinks somebody knows something - on StockTwits.', 'sentiment': 1, 'timestamp': '2018-07-01T00:01:47Z'}, {'message_body': '$AMD reveal yourself!', 'sentiment': 0, 'timestamp': '2018-07-01T00:02:13Z'}, {'message_body': '$AAPL Why the drop? I warren Buffet taking out his position?', 'sentiment': 1, 'timestamp': '2018-07-01T00:03:10Z'}, {'message_body': '$BA bears have 1 reason on 06-29 to pay more attention https://dividendbot.com?s=BA', 'sentiment': -2, 'timestamp': '2018-07-01T

### Length of Data
Now let's look at the number of twits in dataset. Print the number of twits below.

In [3]:
# Print out the number of twits
len(twits['data'])

1548010

### Split Message Body and Sentiment Score

In [4]:
messages = [twit['message_body'] for twit in twits['data']]

# Since the sentiment scores are discrete, we'll scale the sentiments to 0 to 4 for use in our network
sentiments = [twit['sentiment'] + 2 for twit in twits['data']]

## Preprocessing the Data
With our data in hand we need to preprocess our text. These twits are collected by filtering on ticker symbols where these are denoted with a leader $ symbol in the twit itself. For example,

`{'message_body': 'RT @google Our annual look at the year in Google blogging (and beyond) http://t.co/sptHOAh8 $GOOG',
 'sentiment': 0}`

The ticker symbols don't provide information on the sentiment, and they are in every twit, so we should remove them. This twit also has the `@google` username, again not providing sentiment information, so we should also remove it. We also see a URL `http://t.co/sptHOAh8`. Let's remove these too.

The easiest way to remove specific words or phrases is with regex using the `re` module. You can sub out specific patterns with a space:

```python
re.sub(pattern, ' ', text)
```
This will substitute a space with anywhere the pattern matches in the text. Later when we tokenize the text, we'll split appropriately on those spaces.

### Pre-Processing

In [5]:
nltk.download('wordnet')


def preprocess(message):
    """
    This function takes a string as input, then performs these operations: 
        - lowercase
        - remove URLs
        - remove ticker symbols 
        - removes punctuation
        - tokenize by splitting the string on whitespace 
        - removes any single character tokens
    
    Parameters
    ----------
        message : The text message to be preprocessed.
        
    Returns
    -------
        tokens: The preprocessed text into tokens.
    """ 
    # Lowercase the twit message
    text = message.lower()
    
    # Replace URLs with a space in the message
    text = re.sub(r'http\S+', ' ', text)
    
    # Replace ticker symbols with a space. The ticker symbols are any stock symbol that starts with $.
    text = re.sub(r'\$\S+', ' ', text, flags=re.MULTILINE)
    
    # Replace StockTwits usernames with a space. The usernames are any word that starts with @.
    text = re.sub(r'^@\S+', ' ', text, flags=re.MULTILINE)

    # Replace everything not a letter with a space
    text = re.sub(r'[^a-z ]+', ' ', text)
    
    # Tokenize by splitting the string on whitespace into a list of words
    tokens = text.split()
    tokens = [word for word in tokens if len(word) > 1] # remove single character tokens

    # Lemmatize words using the WordNetLemmatizer. You can ignore any word that is not longer than one character.
    wnl = nltk.stem.WordNetLemmatizer()
    tokens = [wnl.lemmatize(text) for text in tokens]
    
    return tokens

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Preprocess All the Twits 
Now we can preprocess each of the twits in our dataset. Apply the function `preprocess` to all the twit messages.

In [6]:
%%time
# TODO Implement
tokenized = [preprocess(msg) for msg in messages]

CPU times: user 2min 44s, sys: 1.23 s, total: 2min 45s
Wall time: 2min 45s


### Bag of Words
Now with all of our messages tokenized, we want to create a vocabulary and count up how often each word appears in our entire corpus. Use the [`Counter`](https://docs.python.org/3.1/library/collections.html#collections.Counter) function to count up all the tokens.

In [7]:
from collections import Counter

#Create a vocabulary by using Bag of words
words = []
for msg in tokenized:
    words.extend(msg)

bow = Counter(words)
print('Bag of words:')
print(list(bow.items())[:10])

Bag of words:
[('great', 17564), ('buy', 76096), ('at', 138454), ('ill', 881), ('wait', 11366), ('staanalystalert', 4072), ('for', 273537), ('jefferies', 1025), ('maintains', 903), ('with', 68856)]


### Frequency of Words Appearing in Message
With our vocabulary, now we'll remove some of the most common words such as 'the', 'and', 'it', etc. These words don't contribute to identifying sentiment and are really common, resulting in a lot of noise in our input. If we can filter these out, then our network should have an easier time learning.

We also want to remove really rare words that show up in a only a few twits. Here you'll want to divide the count of each word by the number of messages. Then remove words that only appear in some small fraction of the messages.

In [8]:
"""
Set the following variables:
    freqs
    low_cutoff
    high_cutoff
    K_most_common
"""
# TODO Implement
total_count = len(words)

# Dictionart that contains the Frequency of words appearing in messages.
# The key is the token and the value is the frequency of that word in the corpus.
freqs = {word: count/total_count for word, count in bow.items()}

# Float that is the frequency cutoff. Drop words with a frequency that is lower or equal to this number.
low_cutoff = 5e-7

# Integer that is the cut off for most common words. Drop words that are the `high_cutoff` most common words.
high_cutoff = 15

# The k most common words in the corpus. Use `high_cutoff` as the k.
K_most_common = dict(bow.most_common(high_cutoff))

filtered_words = [word for word in freqs if (freqs[word] > low_cutoff and word not in K_most_common)]

print('K_most_common:')
print(K_most_common)
print('\nLength of filtered_words:', len(filtered_words))

K_most_common:
{'the': 398813, 'to': 379489, 'is': 284868, 'for': 273537, 'on': 241665, 'of': 211344, 'and': 208474, 'in': 205311, 'this': 203552, 'it': 193499, 'at': 138454, 'will': 128180, 'up': 121574, 'are': 101424, 'you': 94277}

Length of filtered_words: 21708


### Updating Vocabulary by Removing Filtered Words
Let's creat three variables that will help with our vocabulary.

In [9]:
%%time
def create_lookup_tables(words):
    '''
    Create lookup tables for vocabulary
    param words: Input list of words
    return: Two dictionaries, vocab_to_int, int_to_vocab
    '''
    ## Build a dictionary that maps words to integers
    counts = Counter(words)
    vocab = sorted(counts, key=counts.get, reverse=True)
    vocab_to_int = {word: ii for ii, word in enumerate(vocab, 1)}
    int_to_vocab = {ii: word for word, ii in vocab_to_int.items()}
    
    return vocab_to_int, int_to_vocab


# vocab: A dictionary for the `filtered_words`. The key is the word and value is an id that represents the word. 
# id2vocab: Reverse of the `vocab` dictionary. The key is word id and value is the word. 
vocab, id2vocab = create_lookup_tables(filtered_words)

# tokenized with the words not in `filtered_words` removed.
filtered = []
for tokens in tqdm(tokenized):
    message = []
    for token in tokens:
        if token in vocab:
            message.append(token)
    filtered.append(message)

100%|██████████| 1548010/1548010 [00:11<00:00, 133180.31it/s]

CPU times: user 11.5 s, sys: 164 ms, total: 11.7 s
Wall time: 11.6 s


### Balancing the classes
Let's do a few last pre-processing steps. If we look at how our twits are labeled, we'll find that 50% of them are neutral. This means that our network will be 50% accurate just by guessing 0 every single time. To help our network learn appropriately, we'll want to balance our classes.
That is, make sure each of our different sentiment scores show up roughly as frequently in the data.

What we can do here is go through each of our examples and randomly drop twits with neutral sentiment. What should be the probability we drop these twits if we want to get around 20% neutral twits starting at 50% neutral? We should also take this opportunity to remove messages with length 0.

In [10]:
balanced = {'messages': [], 'sentiments':[]}

n_neutral = sum(1 for each in sentiments if each == 2)
N_examples = len(sentiments)
keep_prob = (N_examples - n_neutral)/4/n_neutral

for idx, sentiment in enumerate(sentiments):
    message = filtered[idx]
    if len(message) == 0:
        # skip this message because it has length zero
        continue
    elif sentiment != 2 or random.random() < keep_prob:
        balanced['messages'].append(message)
        balanced['sentiments'].append(sentiment) 

If you did it correctly, you should see the following result 

In [11]:
n_neutral = sum(1 for each in balanced['sentiments'] if each == 2)
N_examples = len(balanced['sentiments'])
n_neutral/N_examples

0.19448658602677937

Finally let's convert our tokens into integer ids which we can pass to the network.

In [12]:
token_ids = [[vocab[word] for word in message] for message in balanced['messages']]
sentiments = balanced['sentiments']

## Neural Network
Now we have our vocabulary which means we can transform our tokens into ids, which are then passed to our network. So, let's define the network now!

Here is a nice diagram showing the network we'd like to build: 

#### Embed -> RNN -> Dense -> Softmax
### Implement the text classifier
Before we build text classifier, if you remember from the other network that you built in  "Sentiment Analysis with an RNN"  exercise  - which there, the network called " SentimentRNN", here we named it "TextClassifer" - consists of three main parts: 1) init function `__init__` 2) forward pass `forward`  3) hidden state `init_hidden`. 

This network is pretty similar to the network you built expect in the  `forward` pass, we use softmax instead of sigmoid. The reason we are not using sigmoid is that the output of NN is not a binary. In our network, sentiment scores have 5 possible outcomes. We are looking for an outcome with the highest probability thus softmax is a better choice.

In [13]:
# First checking if GPU is available
train_on_gpu=torch.cuda.is_available()

if(train_on_gpu):
    print('Training on GPU.')
else:
    print('No GPU available, training on CPU.')

Training on GPU.


In [14]:
class TextClassifier(nn.Module):
    def __init__(self, vocab_size, embed_size, lstm_size, output_size,
                 lstm_layers=1, dropout=0.1, train_on_gpu=torch.cuda.is_available()):
        """
        Initialize the model by setting up the layers.
        
        Parameters
        ----------
            vocab_size : The vocabulary size.
            embed_size : The embedding layer size.
            lstm_size : The LSTM layer size.
            output_size : The output size.
            lstm_layers : The number of LSTM layers.
            dropout : The dropout probability.
        """
        super().__init__()
        self.vocab_size = vocab_size
        self.embed_size = embed_size
        self.lstm_size = lstm_size
        self.output_size = output_size
        self.lstm_layers = lstm_layers
        self.dropout = dropout
        self.train_on_gpu = train_on_gpu
        
        # Setup embedding and LSTM layer
        self.embedding = nn.Embedding(self.vocab_size, self.embed_size)
        self.lstm = nn.LSTM(self.embed_size, self.lstm_size, self.lstm_layers, 
                            dropout=self.dropout, batch_first=False)
        
        # dropout layer
        self.Dropout = nn.Dropout(self.dropout)
        
        # Setup additional layers
        self.fc = nn.Linear(self.lstm_size, self.output_size)
        self.log_softmax = nn.LogSoftmax(dim=1)
    
    
    def forward(self, nn_input, hidden_state):
        """
        Perform a forward pass of our model on nn_input.
        
        Parameters
        ----------
            nn_input : The batch of input to the NN.
            hidden_state : The LSTM hidden state.

        Returns
        -------
            logps: log softmax output
            hidden_state: The new hidden state.

        """
        batch_size = nn_input.size(0)

        # embeddings and lstm_out
        nn_input = nn_input.long()
        embeds = self.embedding(nn_input)
        lstm_out, hidden_state = self.lstm(embeds, hidden_state)
    
        # stack up lstm outputs
        #lstm_out = lstm_out.contiguous().view(-1, self.lstm_size)
        lstm_out = lstm_out[-1,:,:]
        
        # dropout and fully-connected layer
        out = self.Dropout(lstm_out)
        out = self.fc(out)
        
        # softmax function
        logps = self.log_softmax(out)
        
        # return last sigmoid output and hidden state
        return logps, hidden_state


    def init_hidden(self, batch_size):
        """ 
        Initializes hidden state
        
        Parameters
        ----------
            batch_size : The size of batches.
        
        Returns
        -------
            hidden_state
            
        """
        # Create two new tensors with sizes n_layers(lstm_layers) x batch_size x hidden_dim(lstm_size),
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        hidden = (weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_(),
                  weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_())
        
        return hidden

### View Model

In [15]:
model = TextClassifier(len(vocab), 10, 6, 5, dropout=0.1, lstm_layers=2)
print(model)

print()
model.embedding.weight.data.uniform_(-1, 1)

input_ = torch.randint(0, 1000, (5, 4), dtype=torch.int64)
hidden = model.init_hidden(4)

logps, _ = model.forward(input_, hidden)
print(logps)

TextClassifier(
  (embedding): Embedding(21708, 10)
  (lstm): LSTM(10, 6, num_layers=2, dropout=0.1)
  (Dropout): Dropout(p=0.1)
  (fc): Linear(in_features=6, out_features=5, bias=True)
  (log_softmax): LogSoftmax()
)

tensor([[-1.8404, -1.4322, -1.2857, -2.0641, -1.6141],
        [-1.8255, -1.4452, -1.2854, -2.0609, -1.6130],
        [-1.8216, -1.4444, -1.2910, -2.0701, -1.6036],
        [-1.8344, -1.4124, -1.3006, -2.0860, -1.6085]])


## Training
### DataLoaders and Batching
Now we should build a generator that we can use to loop through our data. It'll be more efficient if we can pass our sequences in as batches. Our input tensors should look like `(sequence_length, batch_size)`. So if our sequences are 40 tokens long and we pass in 25 sequences, then we'd have an input size of `(40, 25)`.

If we set our sequence length to 40, what do we do with messages that are more or less than 40 tokens? For messages with fewer than 40 tokens, we will pad the empty spots with zeros. We should be sure to **left** pad so that the RNN starts from nothing before going through the data. If the message has 20 tokens, then the first 20 spots of our 40 long sequence will be 0. If a message has more than 40 tokens, we'll just keep the first 40 tokens.

In [16]:
def dataloader(messages, labels, sequence_length=30, batch_size=32, shuffle=False):
    """ 
    Build a dataloader.
    """
    if shuffle:
        indices = list(range(len(messages)))
        random.shuffle(indices)
        messages = [messages[idx] for idx in indices]
        labels = [labels[idx] for idx in indices]

    total_sequences = len(messages)

    for ii in range(0, total_sequences, batch_size):
        batch_messages = messages[ii: ii+batch_size]
        
        # First initialize a tensor of all zeros
        batch = torch.zeros((sequence_length, len(batch_messages)), dtype=torch.int64)
        for batch_num, tokens in enumerate(batch_messages):
            token_tensor = torch.tensor(tokens)
            # Left pad!
            start_idx = max(sequence_length - len(token_tensor), 0)
            batch[start_idx:, batch_num] = token_tensor[:sequence_length]
        
        label_tensor = torch.tensor(labels[ii: ii+len(batch_messages)])
        
        yield batch, label_tensor

### Training and  Validation
With our data in nice shape, we'll split it into training and validation sets.

In [17]:
"""
Split data into training and validation datasets. Use an appropriate split size.
The features are the `token_ids` and the labels are the `sentiments`.
"""   
split_frac = 0.8

## split data into training, validation, and test data (features and labels, x and y)
split_idx = int(len(token_ids)*split_frac)

train_features = token_ids[:split_idx]
valid_features = token_ids[split_idx:]
train_labels = sentiments[:split_idx]
valid_labels = sentiments[split_idx:]

In [18]:
text_batch, labels = next(iter(dataloader(train_features, train_labels, sequence_length=20, batch_size=64)))
model = TextClassifier(len(vocab)+1, 200, 128, 5, dropout=0.)
hidden = model.init_hidden(64)
logps, hidden = model.forward(text_batch, hidden)

### Training
It's time to train the neural network!

In [19]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = TextClassifier(len(vocab)+1, 1024, 512, 5, lstm_layers=2, dropout=0.2)
model.embedding.weight.data.uniform_(-1, 1)
model.to(device)

TextClassifier(
  (embedding): Embedding(21709, 1024)
  (lstm): LSTM(1024, 512, num_layers=2, dropout=0.2)
  (Dropout): Dropout(p=0.2)
  (fc): Linear(in_features=512, out_features=5, bias=True)
  (log_softmax): LogSoftmax()
)

In [20]:
%%time
"""
Train your model with dropout. Make sure to clip your gradients.
Print the training loss, validation loss, and validation accuracy for every 100 steps.
"""

epochs = 3
batch_size = 1024
learning_rate = 0.001

# gradient clipping
clip=5

# printing and optimiser
print_every = 100
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# move model to GPU, if available
if(train_on_gpu):
    model.cuda()
model.train()

# train for number of epochs
for epoch in tqdm(range(epochs), 'Epoch Progress: '):
    print('Starting epoch {}'.format(epoch + 1))
    steps = 0
    train_loader = dataloader(train_features, train_labels, batch_size=batch_size,
                          sequence_length=20, shuffle=True)
    
    for text_batch, labels in train_loader:
        steps += 1
        # initialize hidden state
        hidden = model.init_hidden(labels.shape[0])
        
        # Set Device
        text_batch, labels = text_batch.to(device), labels.to(device)
        for each in hidden:
            each.to(device)
        
        # zero accumulated gradients
        model.zero_grad()
        
        # Train Model
        output, hidden = model(text_batch, hidden)
        
        # calculate the loss and perform backprop
        loss = criterion(output.squeeze(), labels)
        loss.backward()
        
        # clip gradient
        nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        
        if steps % print_every == 0:
            valid_loader = dataloader(valid_features, valid_labels, batch_size=batch_size,
                                      sequence_length=20, shuffle=True)
            # Get validation loss
            val_losses = []
            num_correct = 0
            model.eval()
            
            # Implement: Validation
            for inputs, labels in valid_loader:
                # Creating new variables for the hidden state, otherwise
                # we'd backprop through the entire training history
                val_h = model.init_hidden(labels.shape[0])
                
                # move to device
                inputs, labels = inputs.to(device), labels.to(device)
                for each in val_h:
                    each.to(device)
                
                output, val_h = model(inputs, val_h)
                val_loss = criterion(output.squeeze(), labels)
                val_losses.append(val_loss.item())
                
                # convert output probabilities to predicted class (0 or 1)
                top_val, top_class = torch.exp(output).topk(1)

                # compare predictions to true label
                num_correct += torch.sum(top_class.squeeze()==labels)
            
            # switch model into training mode
            model.train()
            
            # Implement: Print metrics
            print("Epoch: {}/{}...".format(epoch+1, epochs),
                  "Step: {}...".format(steps),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)),
                  "Validation Accuracy: {:.6f}".format(num_correct.data.cpu().numpy()/len(valid_labels)))

Epoch Progress: :   0%|          | 0/3 [00:00<?, ?it/s]

Starting epoch 1
Epoch: 1/3... Step: 100... Loss: 0.988322... Val Loss: 0.960567 Validation Accuracy: 0.627715
Epoch: 1/3... Step: 200... Loss: 0.826865... Val Loss: 0.864884 Validation Accuracy: 0.662602
Epoch: 1/3... Step: 300... Loss: 0.825402... Val Loss: 0.823507 Validation Accuracy: 0.678074
Epoch: 1/3... Step: 400... Loss: 0.793903... Val Loss: 0.787498 Validation Accuracy: 0.697266
Epoch: 1/3... Step: 500... Loss: 0.760425... Val Loss: 0.775675 Validation Accuracy: 0.699898
Epoch: 1/3... Step: 600... Loss: 0.781411... Val Loss: 0.759587 Validation Accuracy: 0.709426
Epoch: 1/3... Step: 700... Loss: 0.719356... Val Loss: 0.744823 Validation Accuracy: 0.716322
Epoch: 1/3... Step: 800... Loss: 0.726486... Val Loss: 0.734707 Validation Accuracy: 0.719663


Epoch Progress: :  33%|███▎      | 1/3 [08:18<16:37, 498.65s/it]

Starting epoch 2
Epoch: 2/3... Step: 100... Loss: 0.695160... Val Loss: 0.741830 Validation Accuracy: 0.715972
Epoch: 2/3... Step: 200... Loss: 0.672906... Val Loss: 0.739910 Validation Accuracy: 0.716822
Epoch: 2/3... Step: 300... Loss: 0.688603... Val Loss: 0.729696 Validation Accuracy: 0.721120
Epoch: 2/3... Step: 400... Loss: 0.659033... Val Loss: 0.724362 Validation Accuracy: 0.721911
Epoch: 2/3... Step: 500... Loss: 0.660976... Val Loss: 0.721973 Validation Accuracy: 0.724456
Epoch: 2/3... Step: 600... Loss: 0.724399... Val Loss: 0.717994 Validation Accuracy: 0.724408
Epoch: 2/3... Step: 700... Loss: 0.630936... Val Loss: 0.719191 Validation Accuracy: 0.722722
Epoch: 2/3... Step: 800... Loss: 0.654624... Val Loss: 0.717053 Validation Accuracy: 0.725053


Epoch Progress: :  67%|██████▋   | 2/3 [16:40<08:19, 499.72s/it]

Starting epoch 3
Epoch: 3/3... Step: 100... Loss: 0.611331... Val Loss: 0.733839 Validation Accuracy: 0.722703
Epoch: 3/3... Step: 200... Loss: 0.626628... Val Loss: 0.740037 Validation Accuracy: 0.722795
Epoch: 3/3... Step: 300... Loss: 0.609206... Val Loss: 0.731765 Validation Accuracy: 0.721950
Epoch: 3/3... Step: 400... Loss: 0.634003... Val Loss: 0.728251 Validation Accuracy: 0.722106
Epoch: 3/3... Step: 500... Loss: 0.618748... Val Loss: 0.727275 Validation Accuracy: 0.723116
Epoch: 3/3... Step: 600... Loss: 0.633161... Val Loss: 0.726788 Validation Accuracy: 0.722591
Epoch: 3/3... Step: 700... Loss: 0.567545... Val Loss: 0.730325 Validation Accuracy: 0.722922
Epoch: 3/3... Step: 800... Loss: 0.640411... Val Loss: 0.722771 Validation Accuracy: 0.726559


Epoch Progress: : 100%|██████████| 3/3 [24:59<00:00, 499.45s/it]

CPU times: user 17min 21s, sys: 7min 37s, total: 24min 58s
Wall time: 24min 59s


## Making Predictions
### Prediction 
Okay, now that you have a trained model, try it on some new twits and see if it works appropriately. Remember that for any new text, you'll need to preprocess it first before passing it to the network. Implement the `predict` function to generate the prediction vector from a message.

In [21]:
def predict(text, model, vocab):
    """ 
    Make a prediction on a single sentence.

    Parameters
    ----------
        text : The string to make a prediction on.
        model : The model to use for making the prediction.
        vocab : Dictionary for word to word ids. The key is the word and the value is the word id.

    Returns
    -------
        pred : Prediction vector
    """ 
    # tokenise and put model in evaluation mode
    model.eval()
    tokens = preprocess(text)
    
    # Filter non-vocab words
    tokens = [val for val in tokens if val in vocab]
    # Convert words to ids
    tokens = [vocab[token] for token in tokens]
        
    # Adding a batch dimension
    text_input = torch.from_numpy(np.asarray(tokens)).view(-1,1)
    
    # Get the NN output
    hidden = model.init_hidden(1)
    for each in hidden:
        each.to(device)
    
    logps, _ = model.forward(text_input, hidden)
    
    # Take the exponent of the NN output to get a range of 0 to 1 for each label.
    pred = torch.exp(logps)
    
    # this is for getting only the top output
    #pred = logps.topk(1)[1].item()
    print(f"Uncertainty of prediction: {((1 - torch.exp(logps.topk(1)[0])).item() * 100):.2f}%")
    
    return pred

In [22]:
text = "Google is working on self driving cars, I'm bullish on $goog"
model.to("cpu")
predict(text, model, vocab)

Uncertainty of prediction: 18.75%


tensor([[ 0.0002,  0.0072,  0.0062,  0.8125,  0.1740]])

### Questions: What is the prediction of the model? What is the uncertainty of the prediction?

Based on the labels vector, the sentiment which is most dominant is __4-positive__(3 is neutral). This is with 81.2% confidence. The uncertainty of this prediction (total of rest classes) is 11.8%.

Now we have a trained model and we can make predictions. We can use this model to track the sentiments of various stocks by predicting the sentiments of twits as they are coming in. Now we have a stream of twits. For each of those twits, pull out the stocks mentioned in them and keep track of the sentiments. Remember that in the twits, ticker symbols are encoded with a dollar sign as the first character, all caps, and 2-4 letters, like $AAPL. Ideally, you'd want to track the sentiments of the stocks in your universe and use this as a signal in your larger model(s).

## Testing
### Load the Data 

In [23]:
with open(os.path.join('..', '..', 'data', 'project_6_stocktwits', 'test_twits.json'), 'r') as f:
    test_data = json.load(f)

### Twit Stream

In [24]:
def twit_stream():
    for twit in test_data['data']:
        yield twit

next(twit_stream())

{'message_body': '$JWN has moved -1.69% on 10-31. Check out the movement and peers at  https://dividendbot.com?s=JWN',
 'timestamp': '2018-11-01T00:00:05Z'}

Using the `prediction` function, let's apply it to a stream of twits.

In [25]:
def score_twits(stream, model, vocab, universe):
    """ 
    Given a stream of twits and a universe of tickers, return sentiment scores for tickers in the universe.
    """
    for twit in stream:

        # Get the message text
        text = twit['message_body']
        symbols = re.findall('\$[A-Z]{2,4}', text)
        score = predict(text, model, vocab)

        for symbol in symbols:
            if symbol in universe:
                yield {'symbol': symbol, 'score': score, 'timestamp': twit['timestamp']}

In [26]:
universe = {'$BBRY', '$AAPL', '$AMZN', '$BABA', '$YHOO', '$LQMT', '$FB', '$GOOG', '$BBBY', '$JNUG', '$SBUX', '$MU'}
score_stream = score_twits(twit_stream(), model, vocab, universe)

next(score_stream)

Uncertainty of prediction: 0.49%
Uncertainty of prediction: 34.11%
Uncertainty of prediction: 49.86%


{'symbol': '$AAPL',
 'score': tensor([[ 0.1057,  0.1268,  0.1263,  0.1399,  0.5014]]),
 'timestamp': '2018-11-01T00:00:18Z'}

That's it. You have successfully built a model for sentiment analysis! 

## Submission
Now that you're done with the project, it's time to submit it. Click the submit button in the bottom right. One of our reviewers will give you feedback on your project with a pass or not passed grade. You can continue to the next section while you wait for feedback.